In [1]:
import torch
from torch.autograd import Variable
import shap
import numpy
import pandas
import argparse
import os 
from typing import Any, Dict, List
import pandas as pd
from torch.nn import MSELoss, L1Loss
import numpy as np
import json 
import random
from tqdm import tqdm
from pathlib import Path
from scipy.stats import binned_statistic
import matplotlib.pylab as plt


import torch

import torch_frame
from torch.nn import Linear, Module
from torch_frame.data import Dataset
from torch_frame.data import DataLoader
from torch_frame import TensorFrame, stype
from torch_frame.nn.models.ft_transformer import FTTransformer
from torch.nn import Linear, Module, ModuleList
from typing import Any, Dict, List
from torch import Tensor
import torch 

import torch_frame
from torch_frame import TensorFrame, stype
from torch_frame.data.stats import StatType
from torch_frame.nn.conv import TabTransformerConv, FTTransformerConvs
from torch_frame.nn.encoder import (
    EmbeddingEncoder,
    LinearEncoder,
    LinearEmbeddingEncoder,
    StypeWiseFeatureEncoder,
    MultiCategoricalEmbeddingEncoder,
    TimestampEncoder,
)

import sys
sys.path.append('../modeling/')

from model import TabTransformer, stype_encoder_dict_2, stype_encoder_dict_3
from train import get_args_parser

In [2]:
target = "price"
dense_features = ["LON", 
                "LAT", 
                "building_sqft", 
                "Lot Size", 
                "Year Built", 
                "Garage Number", 
                "Bedrooms", 
                "Baths", 
                "Maintenance Fee", 
                "Tax Rate", 
                "Recent Market Value", 
                "Recent Tax Value", 
                 "index"]
cate = ["status", "Property Type", "County", "Private Pool", "Area Pool"]
time_col = ["date"]
cate_multi = ["Foundation_multiclass", "Garage Types_multiclass", 
            "Roof Type_multiclass", "Pool_feature_multiclass", "floor_type_multiclass", 
            "finance_option_multiclass", "Exterior Type_multiclass", "Style_multiclass"]

add_numerial_features = ["elementary_school_star", "middle_school_star", "high_school_star"]
add_cate_multi = ["school_org"] # ["school_names", "school_grades", "school_org"]
text_embedding_col = ["general_desc_roberta_mean_features"]
img_embedding_col = ["img_emb"]
raw_df = pd.read_pickle("../data/property_structured_12162024.pkl").reset_index(drop=True)
raw_df["index"] = np.arange(len(raw_df))
raw_df["date"] = raw_df["date"].apply(lambda x: x.replace("_", "-"))
dense_features = dense_features + add_numerial_features
cate_multi = cate_multi + add_cate_multi

for feat in dense_features:
    raw_df[feat] = raw_df[feat].fillna(raw_df[feat].mean())

col_to_stype = {}
col_to_stype.update({d: stype.numerical for d in dense_features})
col_to_stype.update({d: stype.timestamp for d in time_col})
col_to_stype.update({target: stype.numerical})
col_to_stype.update({d: stype.categorical for d in cate})
col_to_stype.update({d: stype.multicategorical for d in cate_multi})
raw_df = raw_df[dense_features + cate + cate_multi + time_col + [target]]
stype_encoder_dict = stype_encoder_dict_2

for col in cate_multi:
    raw_df[col] = raw_df[col].apply(lambda d: d if isinstance(d, list) else [])

dataset = Dataset(
    raw_df, 
    col_to_stype=col_to_stype,
    target_col="price"
)
dataset.materialize()
torch.manual_seed(10)
dataset.shuffle()
train_dataset, val_dataset = dataset[:0.8], dataset[0.80:]

In [3]:
train_dataset.col_stats == dataset.col_stats

True

In [4]:
len(dataset), len(train_dataset)

(211960, 169568)

In [5]:
batch_size = 1000 
n_batches = 5
all_datasets = []
for i in range(n_batches):
    d_ = Dataset(raw_df.iloc[batch_size * i : batch_size * (i + 1)], 
                 col_to_stype=col_to_stype, 
                 target_col="price")
    d_.materialize(col_stats=train_dataset.col_stats)
    all_datasets.append(d_)

In [6]:
# all_datasets[0].col_stats["County"], all_datasets[1].col_stats["County"]

In [7]:
# all_datasets[0].tensor_frame.feat_dict

In [8]:
all_datasets[0].col_stats["Property Type"][StatType.COUNT][0]

['single family',
 'townhouse/condo',
 'residential/condo',
 'townhouse',
 'mid/hi-rise condo',
 'country homes/acreage',
 'residential/mobile home',
 'multi-family',
 'residential/manufactured',
 '',
 'residential',
 'residential/lot',
 'residential - modular',
 'farm',
 'farm - manufactured home',
 'lots',
 'land / unimproved land']

In [9]:
all_datasets[1].col_stats["Property Type"][StatType.COUNT][0]

['single family',
 'townhouse/condo',
 'residential/condo',
 'townhouse',
 'mid/hi-rise condo',
 'country homes/acreage',
 'residential/mobile home',
 'multi-family',
 'residential/manufactured',
 '',
 'residential',
 'residential/lot',
 'residential - modular',
 'farm',
 'farm - manufactured home',
 'lots',
 'land / unimproved land']

In [10]:
combine = torch.concat([all_datasets[0].tensor_frame.feat_dict[stype.categorical], 
                        all_datasets[1].tensor_frame.feat_dict[stype.categorical]], 
                       axis=0)
combine.shape

torch.Size([2000, 5])

In [11]:
c = Dataset(raw_df.iloc[0: 2000], 
                 col_to_stype=col_to_stype, 
                 target_col="price")
c.materialize(col_stats=train_dataset.col_stats)
c2 = c.tensor_frame.feat_dict[stype.categorical]

In [12]:
torch.sum(c2 != combine, axis=0)

tensor([0, 0, 0, 0, 0])